In [11]:
import pandas as pd
import numpy as np
import json
import os
import re

from ollama import chat
from ollama import ChatResponse

In [12]:
MODEL_NAME = 'gemma3:12b'

# The embeddings and the dataframe created and saved in Part 1
PATH_TO_EMBEDS = 'compressed_array.npz'
PATH_TO_DF = 'compressed_dataframe.csv.gz'

# Define the API clients

In [3]:
from dotenv import load_dotenv
import os

load_dotenv(os.path.expanduser("~/Desktop/dot-env-api-keys/my-api-keys.env"))

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [4]:
# Tavily web search

from tavily import TavilyClient

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# Helper functions

In [5]:
def initialize_message_history(system_message):

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        }
                    ]

    return message_history

In [6]:
def create_message_history(system_message, user_input):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        },
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]

    return message_history



In [7]:
def initialize_message_history(system_message):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        }
                    ]

    return message_history



In [8]:
# Function to separate the thinking and the response

def process_response(text):
    
    text1 = text.split('</think>')[0]
    text2 = text.split('</think>')[1]
    
    thinking_text = text1 + '</think>'
    response_text = text2.strip()

    return thinking_text, response_text

## System message

In [41]:
chat_agent_system_message = f"""
You are a friendly and helpful research assistant named Serena.

Your knowledge cutoff: August 2024
Current date: August 2025

1. You provide polite answers to simple questions.
If the user's input requires only a simple answer, then output your answer as JSON.

Example session:

Question: Hello. How are you?

You output:

{{
"Answer": "I'm fine, thanks.",
"Status": "DONE"
}}

2. You can also run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
Output your response as a JSON string.

Your available actions are:

find_arxiv_research_papers:
e.g. find_arxiv_research_papers: [list of search keywords and phrases for a RAG search of the ArXiv database.]
Returns research papers from the ArXiv database.

run_web_search:
e.g. run_web_search: [list of search keywords and phrases for a web search]
Returns text content from search results.

You can only call one action at a time.

Example session:

Question: What are the latest techniques for detecting Pneumonia on x-rays using AI?
{{
"Thought": "I should look for relevant research papers in the ArXiv database by using find_arxiv_research_papers.",
"Action": {{"function":"find_arxiv_research_papers", "input": ["Pneumonia detection with AI", "Computer vision", "Object detection"]}},
"Status": "PAUSE"
}}

You will be called again with this:

Observation: <results>A list of research papers and their content</results>

You then output:
{{
"Answer": "Your final report.",
"Status": "DONE"
}}

ALWAYS check your responses to ensure that all required JSON keys and values have been included.
""".strip()


# Set up the LLM

In [13]:
def make_llm_api_call(message_history):

    model_name = MODEL_NAME

    response: ChatResponse = chat(model=model_name, 
                                  messages=message_history,
                                )

    output_text = response['message']['content']

    #thinking_text, response_text = process_response(output_text)

    return output_text


# Example

system_message = "Your name is Molly."
user_message = "What's your name?"

message_history = create_message_history(system_message, user_message)

response_text = make_llm_api_call(message_history)

print(response_text)

#print(response['message']['content'])

My name is Molly! 😊


## Set up the tools



### ArXiv RAG search tool

In [14]:
def run_faiss_search(query_text, top_k):
    
    # Run FAISS exhaustive search
    
    query = [query_text]

    # Vectorize the query string
    query_embedding = sent_model.encode(query)

    # Run the query
    # index_vals refers to the chunk_list index values
    scores, index_vals = faiss_index.search(query_embedding, top_k)
    
    # Get the list of index vals
    index_vals_list = index_vals[0]
    
    return index_vals_list
    

def run_rerank(index_vals_list, query_text):
    
    chunk_list = list(df_data['prepared_text'])

    # Replace the chunk index values with the corresponding strings
    pred_strings_list = [chunk_list[item] for item in index_vals_list]

    # Format the input for the cross encoder
    # The input to the cross_encoder is a list of lists
    # [[query_text, pred_text1], [query_text, pred_text2], ...]

    cross_input_list = []

    for item in pred_strings_list:

        new_list = [query_text, item]

        cross_input_list.append(new_list)


    # Put the pred text into a dataframe
    df = pd.DataFrame(cross_input_list, columns=['query_text', 'pred_text'])

    # Save the orginal index (i.e. df_data index values)
    df['original_index'] = index_vals_list

    # Now, score all retrieved passages using the cross_encoder
    cross_scores = cross_encoder.predict(cross_input_list)

    # Add the scores to the dataframe
    df['cross_scores'] = cross_scores

    # Sort the DataFrame in descending order based on the scores
    df_sorted = df.sort_values(by='cross_scores', ascending=False)
    
    # Reset the index (*This was missed previously*)
    df_sorted = df_sorted.reset_index(drop=True)

    pred_list = []

    for i in range(0,len(df_sorted)):

        text = df_sorted.loc[i, 'pred_text']

        # Get the arxiv id
        # original_index refers to the index values in df_filtered
        original_index = df_sorted.loc[i, 'original_index']
        arxiv_id = df_data.loc[original_index, 'id']
        cat_text = df_data.loc[original_index, 'cat_text']
        title = df_data.loc[original_index, 'title']

        # Crete the link to the research paper pdf
        link_to_pdf = f'https://arxiv.org/pdf/{arxiv_id}'

        item = {
            'arxiv_id': arxiv_id,
            'link_to_pdf': link_to_pdf,
            'cat_text': cat_text,
            'title': title,
            'abstract': text
        }

        pred_list.append(item)

    return pred_list


def print_search_results(pred_list, num_results_to_print):
    
    for i in range(0,num_results_to_print):
        
        pred_dict = pred_list[i]
        
        link_to_pdf = pred_dict['link_to_pdf']
        abstract = pred_dict['abstract']
        cat_text = pred_dict['cat_text']
        title = pred_dict['title']

        print('Title:',title)
        print('Categories:',cat_text)
        print('Abstract:',abstract)
        print('Link to pdf:',link_to_pdf)
        print()
    
   
def run_arxiv_search(query_text, top_k=50):
    
    # Run a faiss greedy search
    pred_index_list = run_faiss_search(query_text, top_k)

    # This returns a list of dicts with length equal to top_k
    pred_list = run_rerank(pred_index_list, query_text)
    
    # Print the results
    #print_search_results(pred_list, num_results_to_print)
    
    return pred_list
    

In [15]:
# Load the compressed array
embeddings = np.load(PATH_TO_EMBEDS)

# Access the array by the name you specified ('my_array' in this case)
embeddings = embeddings['array_data']

embeddings.shape

(2421966, 384)

In [16]:
# Load the compressed DataFrame

df_data = pd.read_csv(PATH_TO_DF, compression='gzip')

print(df_data.shape)

df_data.head()

/var/folders/rn/mv2kfll17l1gh1p7h1nny_940000gn/T/ipykernel_36655/2541750260.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(PATH_TO_DF, compression='gzip')


(2421966, 6)


,id,title,abstract,categories,cat_text,prepared_text
0,704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturbati...,hep-ph,High Energy Physics - Phenomenology,Calculation of prompt diphoton production cros...
1,704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-pe...",math.CO cs.CG,"Combinatorics, Computational Geometry",Sparsity-certifying Graph Decompositions {titl...
2,704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is describe...,physics.gen-ph,General Physics,The evolution of the Earth-Moon system based o...
3,704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle n...,math.CO,Combinatorics,A determinant of Stirling cycle numbers counts...
4,704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\Lam...,math.CA math.FA,"Classical Analysis and ODEs, Functional Analysis",From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...


In [17]:
# Initialize FAISS

import faiss

embed_length = embeddings.shape[1]

faiss_index = faiss.IndexFlatL2(embed_length)

# Add the embeddings to the index
faiss_index.add(embeddings)

faiss_index.is_trained

True

In [18]:
# Initialize sentence_transformers

from sentence_transformers import SentenceTransformer

sent_model = SentenceTransformer("all-MiniLM-L6-v2")

In [19]:
# Initialize the cross_encoder for reranking

from sentence_transformers import CrossEncoder

# We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [20]:
# Example

query_text = """
I want to build an invisibility cloak like the one in Harry Potter.
"""


# RUN THE SEARCH
num_results_to_print = 20 # top_k = 300
pred_list = run_arxiv_search(query_text, top_k=5)

In [21]:
pred_list[0]

{'arxiv_id': 1101.0904,
 'link_to_pdf': 'https://arxiv.org/pdf/1101.0904',
 'cat_text': 'Classical Physics',
 'title': "Harry Potter's Cloak",
 'abstract': 'Harry Potter\'s Cloak {title} The magic "Harry Potter\'s cloak" has been the dream of human beings for really long time. Recently, transformation optics inspired from the advent of metamaterials offers great versatility for manipulating wave propagation at will to create amazing illusion effects. In the present work, we proposed a novel transformation recipe, in which the cloaking shell somehow behaves like a "cloaking lens", to provide almost all desired features one can expect for a real magic cloak. The most exciting feature of the current recipe is that an object with arbitrary characteristics (e.g., size, shape or material properties) can be invisibilized perfectly with positive-index materials, which significantly benefits the practical realization of a broad-band cloaking device fabricated with existing materials. Moreover, 

### Tavily web search

In [22]:
def run_tavily_search(query, num_results=10):

    """
    Uses the Tavily API to run a web search
    Args:
        query (str): The user query
        num_results (int): Num search results
    Returns:
        tav_response (json string): The search results in json format
    """

    # For basic search:
    tav_response = tavily_client.search(query=query, max_results=num_results)

    return tav_response


# Example

query = "Who is the current UK Prime Minister?"

results = run_tavily_search(query, num_results=2)

print(results)

{'query': 'Who is the current UK Prime Minister?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.gov.uk/government/ministers/prime-minister', 'title': 'Prime Minister - GOV.UK', 'content': 'Current role holder. The Rt Hon Sir Keir Starmer KCB KC MP. Sir Keir Starmer became Prime Minister on 5 July 2024. Education. Keir attended Reigate Grammar', 'score': 0.8810534, 'raw_content': None}, {'url': 'https://en.wikipedia.org/wiki/Prime_Minister_of_the_United_Kingdom', 'title': 'Prime Minister of the United Kingdom - Wikipedia', 'content': "Prime Minister of the United Kingdom ; Incumbent Keir Starmer. since 5 July 2024 ; Government of the United Kingdom · Prime Minister's Office", 'score': 0.87782305, 'raw_content': None}], 'response_time': 1.33, 'request_id': 'f119b82e-a400-4762-8cdf-2190798e975d'}


# Set up the Agents

In [24]:
def run_chat_agent(message_history):

    print("---CHAT AGENT---")

    # Prompt the llm
    response = make_llm_api_call(message_history)

    response = response.replace('```json', '')
    response = response.replace('```', '')
    response = response.strip()

    return response


# Example

user_message = "Hello"

message_history = create_message_history(chat_agent_system_message, user_message)

response_text = run_chat_agent(message_history)

print(response_text)

#print(response['message']['content'])

---CHAT AGENT---
{
"Answer": "Hello! How can I help you today?",
"Status": "DONE"
}


In [25]:
def run_router_agent(llm_response):

    """
    Route to web search or not.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """

    print("---ROUTER AGENT---")

    # Extract the status
    json_response = json.loads(llm_response)
    status = json_response['Status']
    
    print("Status:", status)

    if status == 'PAUSE':
        print("Route: to_research_agent")
        return "to_research_agent"

    else:
        print("Route: to_final_answer")
        return "to_final_answer"
            



# Example

user_message = "hello"

message_history = create_message_history(chat_agent_system_message, user_message)

# Prompt the chat_agent
response = run_chat_agent(message_history)

# Run router_agent
route = run_router_agent(response)

print(route)

---CHAT AGENT---
---ROUTER AGENT---
Status: DONE
Route: to_final_answer
to_final_answer


In [26]:
def run_research_agent(llm_response):

    print("---RESEARCH AGENT---")

    # Extract the status
    json_response = json.loads(llm_response)
    action_dict = json_response['Action']
    func_to_run = action_dict['function']
    func_input_list = action_dict['input']
    
    answer_list = []

    if func_to_run == "find_arxiv_research_papers":
        for search_query in func_input_list:
            answer = run_arxiv_search(search_query, top_k=5)
            answer_list.append(answer)
    else:
        for search_query in func_input_list:
            answer = run_tavily_search(search_query, num_results=5)
            answer_list.append(answer)

    print("func_to_run:", func_to_run)
    print("func_arg:", func_input_list)
    print("Output:", answer_list)

    return answer_list



# Example

user_message = "Has OpenAi released any new open source models?"

message_history = create_message_history(chat_agent_system_message, user_message)

# Prompt the chat_agent
response = run_chat_agent(message_history)

# Run router_agent
route = run_router_agent(response)


if route == "to_research_agent":
    answer = run_research_agent(response)

    # Update message history
    #message = {"role": "user", "content": f"Observation: {answer}"}
    #message_history.append(message)



---CHAT AGENT---
---ROUTER AGENT---
Status: PAUSE
Route: to_research_agent
---RESEARCH AGENT---
func_to_run: run_web_search
func_arg: ['OpenAI open source models', 'OpenAI released open source models', 'OpenAI open source AI models']
Output: [{'query': 'OpenAI open source models', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://openai.com/open-models/', 'title': 'Open models by OpenAI', 'content': '*   ChatGPT(opens in a new window) *   Sora(opens in a new window) *   API Platform(opens in a new window) *   Open Models  *   Sora Log in(opens in a new window) Open models by OpenAI Advanced open-weight reasoning models to customize for any use case and run anywhere. Image 1 -0.55 0.16 0.09 0.10 -0.58 ### gpt-oss-120b A large open model designed to run in data centers and on high-end desktops and laptops. Start building(opens in a new window) (opens in a new window)Image 2 -0.55 0.16 0.09 0.10 -0.58 ### gpt-oss-20b A medium-sized open model that can

In [29]:
def check_output_type(output):
    try:
        json.loads(output)
        return "is_json"
    except json.JSONDecodeError:
        return "is_plain_text"

In [27]:
def run_final_answer_agent(llm_response):

    # Check if the output is JSON
    output_type = check_output_type(llm_response)

    print("---FINAL ANSWER AGENT---")

    if output_type == 'is_json':

        json_response = json.loads(llm_response)
        final_answer = json_response['Answer']
    
        print("Final answer:", final_answer)

    # The model has ouput plain text that's
    # not being formatted as per the system message.
    else:
        print(llm_response)


# Run the system

In [30]:

user_input = "What is the current year?"

message_history = create_message_history(chat_agent_system_message, user_input)

for i in range(0,10):

    # Prompt the chat_agent
    llm_response = run_chat_agent(message_history)
    
    # Update message history
    message = {"role": "assistant", "content": llm_response}
    message_history.append(message)

    # Run router_agent
    route = run_router_agent(llm_response)


    if route == "to_research_agent":

        answer = run_research_agent(llm_response)
        
        user_input = f"Observation: {answer}"
        message = {"role": "user", "content": user_input}
        message_history.append(message)

    else:

        run_final_answer_agent(llm_response)

        break



---CHAT AGENT---
---ROUTER AGENT---
Status: DONE
Route: to_final_answer
---FINAL ANSWER AGENT---
Final answer: The current year is 2025.


# Run a chat loop

In [42]:
# NOTES
# 1. The model thinking is being printed out.


# Initialize the message history
message_history = initialize_message_history(chat_agent_system_message)

while True:

    print()
    print("==========")
    user_input = input("Enter something (or 'q' to quit): ")
    print("==========")

    # Update message history
    message = {"role": "user", "content": user_input}
    message_history.append(message)
    
    i = i + 1

    if user_input.lower() == 'q':
        print("Exiting the loop. Goodbye!")
        break  # Exit the loop


    for i in range(0,10):
           
        llm_response = run_chat_agent(message_history)
        
        # Update message history
        message = {"role": "assistant", "content": llm_response}
        message_history.append(message)

        # Run router_agent
        route = run_router_agent(llm_response)
            

        if route == "to_research_agent":

            answer = run_research_agent(llm_response)
            
            user_input = f"Observation: {answer}"
            message = {"role": "user", "content": user_input}
            message_history.append(message)

        else:

            run_final_answer_agent(llm_response)

            break



Enter something (or 'q' to quit):  Hi Serena


---CHAT AGENT---
---ROUTER AGENT---
Status: DONE
Route: to_final_answer
---FINAL ANSWER AGENT---
Final answer: Hello! I'm doing well, thank you for asking. How can I help you today?



Enter something (or 'q' to quit):  q


Exiting the loop. Goodbye!


In [43]:
message_history

[{'role': 'system',
  'content': 'You are a friendly and helpful research assistant named Serena.\n\nYour knowledge cutoff: August 2024\nCurrent date: August 2025\n\n1. You provide polite answers to simple questions.\nIf the user\'s input requires only a simple answer, then output your answer as JSON.\n\nExample session:\n\nQuestion: Hello. How are you?\n\nYou output:\n\n{\n"Answer": "I\'m fine, thanks.",\n"Status": "DONE"\n}\n\n2. You can also run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop, you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\nOutput your response as a JSON string.\n\nYour available actions are:\n\nfind_arxiv_research_papers:\ne.g. find_arxiv_research_papers: [list of search keywords and phrases for a RAG search of the ArXiv database.]\nReturns research paper

In [40]:


print(llm_response)

{
  "query": "Qwen models 2025",
  "response": {
    "summary": "The Qwen models have seen significant development in 2025. Key highlights include the release of Qwen2.5-Max, Qwen2.5-VL (capable of analyzing charts, extracting data from scans, and understanding long videos), and advancements in Qwen3 (including versions with long-context understanding and enhanced capabilities).  Qwen2.5-VL is available for testing and download, while Qwen2.5-Max can be accessed through Qwen Chat.  Qwen models are actively being improved, with a focus on capabilities like controlling PCs and phones, and improved performance across various benchmarks. The models also have some restrictions on topics due to their origin.",
    "key_releases": [
      "Qwen2.5-Max: Large-scale MoE model, available in Qwen Chat.",
      "Qwen2.5-VL:  Analyzes charts, extracts data, understands long videos.",
      "Qwen3:  Long-context understanding, enhanced capabilities (e.g., Qwen3-30B-A3B-Thinking-2507)."
    ],
    "a